# Creating tensors with `tf.Variable`

In [3]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf

In [4]:
#creating tf.constant(unchangable) and tf.Variable(changable)
changable_tensor=tf.Variable([2,5,6])
unchangabel_tensor=tf.constant([2,6,2])

2025-10-08 07:35:45.853952: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [5]:
changable_tensor

<tf.Variable 'Variable:0' shape=(3,) dtype=int32, numpy=array([2, 5, 6], dtype=int32)>

In [6]:
#lets try to change one of the elements of the changable of tensor
changable_tensor[0]=[3]

TypeError: 'ResourceVariable' object does not support item assignment

In [ ]:
#lets try to change one of the elements of the changabel tensor with .assign()
changable_tensor[0].assign(3)
print(changable_tensor)

<tf.Variable 'Variable:0' shape=(3,) dtype=int32, numpy=array([3, 5, 6], dtype=int32)>


In [ ]:
#lets try to change the elements of unchangable tensor with .assign()
unchangabel_tensor[0].assign(33)
print(unchangable_tensor)

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'

# Creating RandomTensor with TensorFlow

In [9]:
#create random tensor
random_tensor=tf.random.normal(shape=(3,2))
print(random_tensor)

tf.Tensor(
[[ 1.2585617  -1.4151684 ]
 [ 0.5874982  -0.57268715]
 [ 0.8059718   0.33799234]], shape=(3, 2), dtype=float32)


In [ ]:
#creating two random(but the same) tensor
random_1=tf.random.Generator.from_seed(43) #set seed for reproducibility
random_1=random_1.normal(shape=(3,2))
random_2=tf.random.Generator.from_seed(43)
random_2=random_2.normal(shape=(3,2))
print(random_1)
print(random_2)
print(random_1==random_2)

tf.Tensor(
[[-0.23193763 -1.8107855 ]
 [ 0.09988727 -0.50998646]
 [-0.7535805  -0.57166284]], shape=(3, 2), dtype=float32)
tf.Tensor(
[[-0.23193763 -1.8107855 ]
 [ 0.09988727 -0.50998646]
 [-0.7535805  -0.57166284]], shape=(3, 2), dtype=float32)
tf.Tensor(
[[ True  True]
 [ True  True]
 [ True  True]], shape=(3, 2), dtype=bool)


# Shuffle the order of elements in the tensor `tf.random.shuffle`

#### In machine learning / deep learning, we often shuffle data before training because:

### 1.Avoids learning order bias
##### If the data is ordered (for example, all class A samples first, then all class B), the model might start learning patterns that depend on the order instead of the actual data relationships.

### 2.Improves generalization
##### Shuffling makes the model see a more random mix of examples, helping it generalize better and not overfit to a particular sequence.

### 3.Ensures better gradient updates
##### During mini-batch training, random batches created from shuffled data give more diverse gradients, leading to smoother and faster convergence.

In [18]:
random_3=tf.constant([[3,5,6],[2,1,2],[3,5,6],[9,0,3]])
print('original matrix is')
print(random_3)
print('shuffled of matrix is :')
print(tf.random.shuffle(random_3))

original matrix is
tf.Tensor(
[[3 5 6]
 [2 1 2]
 [3 5 6]
 [9 0 3]], shape=(4, 3), dtype=int32)
shuffled of matrix is :
tf.Tensor(
[[9 0 3]
 [2 1 2]
 [3 5 6]
 [3 5 6]], shape=(4, 3), dtype=int32)


# Types of seeds in TensorFlow

##### 1.If neither the global seed nor the operation seed is set, we get different results for every call to the random op and every re-run of the program:

In [124]:

print(tf.random.uniform([1]))  # generates 'A1'
print(tf.random.uniform([1]))  # generates 'A2'

tf.Tensor([0.39475608], shape=(1,), dtype=float32)
tf.Tensor([0.50503516], shape=(1,), dtype=float32)


#### 2.If the global seed is set but the operation seed is not set, we get different results for every call to the random op, but the same sequence for every re-run of the program:

In [130]:
tf.random.set_seed(1234)
print(tf.random.uniform([1]))  # generates 'A1'
print(tf.random.uniform([1]))  # generates 'A2'

tf.Tensor([0.5380393], shape=(1,), dtype=float32)
tf.Tensor([0.3253647], shape=(1,), dtype=float32)


#### 3.The reason we get 'A2' instead 'A1' on the second call of tf.random.uniform is because the second call uses a different operation seed.

In [132]:
print(tf.random.uniform([1]))  # generates 'A1'
print(tf.random.uniform([1]))  # generates 'A2'

tf.Tensor([0.5380393], shape=(1,), dtype=float32)
tf.Tensor([0.3253647], shape=(1,), dtype=float32)


#### 4.Note that `tf.function` acts like a re-run of a program in this case. When the global seed is set but operation seeds are not set, the sequence of random numbers are the same for each `tf.function`. For example:


In [136]:
tf.random.set_seed(1234)

@tf.function
def f():
  a = tf.random.uniform([1])
  b = tf.random.uniform([1])
  return a, b

@tf.function
def g():
  a = tf.random.uniform([1])
  b = tf.random.uniform([1])
  return a, b

print(f())  # prints '(A1, A2)'
print(g())  # prints '(A1, A2)'

(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.13047123], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1689806], dtype=float32)>)
(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.13047123], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1689806], dtype=float32)>)


#### 5.If the operation seed is set, we get different results for every call to the random op, but the same sequence for every re-run of the program:

In [165]:
print(tf.random.uniform([1], seed=1))  # generates 'A1'
print(tf.random.uniform([1], seed=4))  # generates 'A2'

tf.Tensor([0.1510979], shape=(1,), dtype=float32)
tf.Tensor([0.3737167], shape=(1,), dtype=float32)


#### 6.Calling tf.random.set_seed will reset any such counters:

In [173]:
tf.random.set_seed(1234)
print(tf.random.uniform([1], seed=1))  # generates 'A1'
print(tf.random.uniform([1], seed=1))  # generates 'A2'
tf.random.set_seed(1234)
print(tf.random.uniform([1], seed=1))  # generates 'A1'
print(tf.random.uniform([1], seed=1))  # generates 'A2'

tf.Tensor([0.1689806], shape=(1,), dtype=float32)
tf.Tensor([0.7539084], shape=(1,), dtype=float32)
tf.Tensor([0.1689806], shape=(1,), dtype=float32)
tf.Tensor([0.7539084], shape=(1,), dtype=float32)


#### 7.The second call of foo returns '(A2, A2)' instead of '(A1, A1)' because tf.random.uniform maintains an internal counter. If you want foo to return '(A1, A1)' every time, use the stateless random ops such as tf.random.stateless_uniform. Also see tf.random.experimental.Generator for a new set of stateful random ops that use external variables to manage their states.



In [180]:
@tf.function
def foo():
  a = tf.random.uniform([1], seed=1)
  b = tf.random.uniform([1], seed=1)
  return a, b
print(foo())  # prints '(A1, A1)'
print(foo())  # prints '(A2, A2)'

@tf.function
def bar():
  a = tf.random.uniform([1])
  b = tf.random.uniform([1])
  return a, b
print(bar())  # prints '(A1, A2)'
print(bar())  # prints '(A3, A4)'

(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1689806], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1689806], dtype=float32)>)
(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.7539084], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.7539084], dtype=float32)>)
(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.13047123], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1689806], dtype=float32)>)
(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.6087816], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.7539084], dtype=float32)>)
